In [1]:
# Based in part on (https://github.com/fneum/spatial-sector/blob/main/notebooks/sankey.ipynb)

import pypsa
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import yaml
import os
from matplotlib.colors import to_rgba

import os
import yaml
import pypsa

import matplotlib as mpl

mpl.rcParams["figure.dpi"] = 150

import pandas as pd
import numpy as np

import os
import warnings

import pypsa
import yaml
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings("ignore")

In [2]:
cm = 1 / 2.54  # centimeters in inches

config_fn = "../config/config.default.yaml"
config = yaml.safe_load(open(config_fn))

copy_networks = False
use_cache = True

In [3]:
# Set colours.
colors = config["plotting"]["tech_colors"]

colors["electricity grid"] = colors["electricity"]
colors["wind"] = colors["onwind"]
colors["ground-sourced ambient"] = colors["ground heat pump"]
colors["air-sourced ambient"] = colors["air heat pump"]
colors["ambient"] = colors["air heat pump"]
colors["co2 atmosphere"] = colors["co2"]
colors["co2 stored"] = colors["co2"]
colors["net co2 emissions"] = colors["co2"]
colors["co2 sequestration"] = colors["co2"]
colors["fossil oil"] = colors["oil"]
colors["fossil gas"] = colors["gas"]
colors["biogas to gas"] = colors["biogas"]
colors["process emissions from feedstocks"] = colors["process emissions"]
colors["biomass"] = colors["biogas"]

colors["renewables"] = "#4491d0"
colors["H2"] = "#5fc983"
colors["gas"] = "#f3a068"
colors["fossil gas"] = "#f3a068"
colors["oil"] = "#a5826c"
colors["fossil oil"] = "#a5826c"
colors["electricity grid"] = "#90e5db"
colors["ambient heat"] = "#c67843"

gas_boilers = [
    "residential rural gas boiler",
    "services rural gas boiler",
    "residential urban decentral gas boiler",
    "services urban decentral gas boiler",
    "urban central gas boiler",
]
for gas_boiler in gas_boilers:
    colors[gas_boiler] = colors["gas boiler"]

colors["urban central gas CHP"] = colors["CHP"]
colors["urban central gas CHP CC"] = colors["CHP"]
colors["urban central solid biomass CHP"] = colors["CHP"]
colors["urban central solid biomass CHP CC"] = colors["CHP"]

In [4]:
def make_connections(n):
    connections = pd.DataFrame(columns=["label", "source", "target", "value"])
    eb = n.statistics.energy_balance(nice_names=False)
    for component, carrier in eb.index.droplevel("bus_carrier").unique():
        c = eb.xs((component, carrier), level=["component", "carrier"])
        c = c.loc[~c.index.str.contains("co2|process emissions")]
        targets = c.loc[c >= 0]
        sources = -c.loc[c < 0]
        if targets.empty and sources.empty:
            continue
        if targets.empty:
            # This is purely a load
            load_bus = sources.index[0]
            connections.loc[len(connections)] = {
                "label": carrier,
                "source": load_bus,
                "target": carrier + " demand",
                "value": sources.loc[load_bus],
            }
            continue
        if sources.empty:
            # This is purely a generator
            gen_bus = targets.index[0]
            connections.loc[len(connections)] = {
                "label": carrier,
                "source": carrier,
                "target": gen_bus,
                "value": targets.loc[gen_bus],
            }
            continue
        
        losses = sources.sum() - targets.sum()

        # For "normal" components, losses are positive
        if losses >= -1e3:
            for source in sources.index:
                connections.loc[len(connections)] = {
                    "label": carrier,
                    "source": source,
                    "target": "losses",
                    "value": -c.loc[source] * losses / sources.sum(),
                }
            for target in targets.index:
                for source in sources.index:
                    connections.loc[len(connections)] = {
                        "label": carrier,
                        "source": source,
                        "target": target,
                        "value": -c.loc[source]
                        * (targets.sum() / sources.sum())
                        * (c.loc[target] / targets.sum()),
                    }
        # For heat pumps, losses are negative and the source is ambient heat
        else:
            # Simply make a connection from the source to each target
            assert(len(sources) == 1)
            source = sources.index[0]
            for target in targets.index:
                connections.loc[len(connections)] = {
                    "label": carrier,
                    "source": source,
                    "target": target,
                    "value": -c.loc[source] * (c.loc[target] / targets.sum()),
                }
            # Add a connection from "ambient heat" to the target containing "heat"
            heat_target = targets.index[targets.index.str.contains("heat|(low|medium)T")][0]
            connections.loc[len(connections)] = {
                "label": carrier,
                "source": "ambient",
                "target": heat_target,
                "value": -losses,
            }
            

    connections.value /= 1e6
    return connections


def prepare_sankey(n, losses=True, agg_flows=False):
    # This function is partially based on code found in https://github.com/fneum/spatial-sector/blob/main/notebooks/sankey.ipynb
    columns = ["label", "source", "target", "value"]

    connections = make_connections(n)

    # aggregation

    connections_pre_agg = connections.copy()

    src_contains = connections.source.str.contains
    trg_contains = connections.target.str.contains

    connections.loc[src_contains("low voltage"), "source"] = "AC"
    connections.loc[trg_contains("low voltage"), "target"] = "AC"
    connections.loc[src_contains("water tank"), "source"] = "water tank"
    connections.loc[trg_contains("water tank"), "target"] = "water tank"
    connections.loc[src_contains("solar thermal"), "source"] = "solar thermal"
    connections.loc[src_contains("battery"), "source"] = "battery"
    connections.loc[trg_contains("battery"), "target"] = "battery"
    connections.loc[src_contains("Li ion"), "source"] = "battery"
    connections.loc[trg_contains("Li ion"), "target"] = "battery"

    connections.loc[src_contains("heat") & ~src_contains("demand"), "source"] = "heat"
    connections.loc[trg_contains("heat") & ~trg_contains("demand"), "target"] = "heat"

    # Aggregate some energy input
    # Wind (onshore, offshore, etc.)
    connections.loc[
        connections.source.isin(
            ["onwind", "offwind-ac", "offwind-dc", "offwind-float"]
        ),
        "source",
    ] = "wind"
    # Hydro, ror
    connections.loc[connections.source.isin(["ror", "hydro"]), "source"] = "hydro"
    # Coal and lignite
    connections.loc[connections.source.isin(["lignite", "coal"]), "source"] = "coal"
    connections.loc[connections.target.isin(["lignite", "coal"]), "target"] = "coal"
    # Rename uranium to nuclear
    connections.loc[connections.source == "uranium", "source"] = "nuclear"
    connections.loc[connections.target == "uranium", "target"] = "nuclear"
    # Rename "ambient" to "ambient heat" as source
    connections.loc[connections.source == "ambient", "source"] = "ambient heat"

    # Aggregate solid biomass and biogas sources
    connections.loc[connections.source.isin(["solid biomass", "biogas"]), "source"] = (
        "biomass"
    )
    connections.loc[connections.target.isin(["solid biomass", "biogas"]), "target"] = (
        "biomass"
    )

    # Remove links with "oil primary" as target
    connections = connections.loc[~(connections.target == "oil primary")]

    # The connection from "gas" to "gas" should have a source of "fossil gas"
    connections.loc[
        (connections.source == "gas") & (connections.target == "gas"), "source"
    ] = "fossil gas"

    # Final trimming
    exo_oil = [
        "shipping oil",
        "kerosene for aviation",
        "naphtha for industry",
        "agriculture machinery oil",
    ]
    connections = connections.loc[
        ~(connections.source == connections.target)
        & ~connections.source.str.contains("co2")
        & ~connections.target.str.contains("co2")
        & ~connections.source.str.contains("emissions")
        & ~connections.target.str.contains("emissions")
        & ~connections.source.str.contains("water tank")
        & ~connections.target.str.contains("water tank")
        & ~((connections.source == "uranium") & (connections.target == "losses"))
        & ~connections.source.isin(
            [
                "gas for industry",
                "coal for industry",
                "solid biomass for industry",
                "shipping methanol",
                "NH3",
            ]
            + exo_oil
        )
    ]

    # Aggregate all renewables
    connections.loc[
        connections.source.isin(["wind", "solar", "hydro", "nuclear"]),
        "source",
    ] = "renewables"

    # Aggregate all green fuel imports; source contains string "green import"
    connections.loc[connections.source.str.contains("green import"), "source"] = (
        "green fuel imports"
    )

    # Remove connections with target "(lowT|mediumT|highT) industry demand"
    connections = connections.loc[
        ~connections.target.str.contains("(lowT|mediumT|highT) industry demand")
    ]

    # Aggregate industry demand
    connections.loc[connections.target.str.contains("industry"), "target"] = "industry"
    connections.loc[connections.target == "NH3", "target"] = "industry"

    # Aggregate agriculture demand
    connections.loc[connections.target.str.contains("agriculture"), "target"] = (
        "agriculture"
    )

    # Same with "shipping (methanol|oil|ammonia|hydrogen|gas) demand"
    connections = connections.loc[
        ~connections.target.str.contains(
            "shipping (methanol|oil|ammonia|hydrogen|gas) demand"
        )
    ]

    # Transportation / shipping demand
    connections.loc[
        connections.target.isin(
            [
                "kerosene for aviation",
                "shipping methanol",
                "shipping oil",
                "shipping ammonia",
                "shipping hydrogen",
                "shipping gas",
            ]
        )
        | connections.target.str.contains("land transport"),
        "target",
    ] = "transport"

    # Need to remove connection with land transport oil source
    connections = connections.loc[
        ~connections.source.str.contains("land transport oil")
    ]

    # Make methanol demand transport instead; remove any connections from methanol
    connections.loc[connections.target == "methanol", "target"] = "transport"
    connections = connections.loc[~(connections.source == "methanol")]

    # Domestic demand
    connections.loc[
        connections.target.str.contains("urban")
        | connections.target.str.contains("rural")
        | (connections.target == "electricity demand"),
        "target",
    ] = "domestic & services"

    # Aggregate industry and agriculture
    connections.loc[connections.target.isin(["industry", "agriculture"]), "target"] = (
        "industry & agriculture"
    )

    connections_pre_agg = connections.copy()

    # Finally aggregate all demands: transport, industry, agriculture, domestic, DAC
    connections.loc[
        connections.target.isin(
            ["transport", "industry & agriculture", "domestic & services", "DAC demand"]
        ),
        "target",
    ] = "other demand"

    where = connections.source == "losses"
    target = connections.loc[where, "target"]
    connections.loc[where, "source"] = target
    connections.loc[where, "target"] = "losses"

    connections.replace("AC", "electricity grid", inplace=True)

    if not losses:
        connections = connections.loc[~(connections.target == "losses")]

    # Remove "heat" is a separate bus and make it a demand
    connections = connections.loc[~(connections.source == "heat")]
    connections.loc[connections.target == "heat", "target"] = "heat demand"

    if agg_flows:
        connections = (
            connections.drop(columns=["label"])
            .groupby(["source", "target"])
            .sum()
            .reset_index()
        )
        connections["label"] = connections.source + " to " + connections.target
        connections = connections[columns]

    # Remove connections with small values
    threshold = 50 if agg_flows else 5
    connections = connections.loc[connections.value > threshold]

    return connections, connections_pre_agg


def plot_sankey(connections, max_elem_height=10000, pad=2, fn=None, title=None):
    # Add an artificial element to the connection, to be located at the far left
    # of the plot in order to fix the scale; a connection from dummy1 to dummy2
    # with a high value. Also add dummy connections to make for the same
    # topology in every plot.
    connections = pd.concat(
        [
            connections,
            pd.DataFrame(
                {
                    "label": "dummy",
                    "source": ["dummy1", "dummy2", "H2", "battery", "electricity grid"],
                    "target": [
                        "dummy2",
                        "renewables",
                        "electricity grid",
                        "electricity grid",
                        "H2",
                    ],
                    "value": [max_elem_height, 1, 1, 1, 1],
                },
            ),
        ]
    )

    labels = np.unique(connections[["source", "target"]])

    positions = {
        "dummy1": (-2, 0.5),
        "dummy2": (-1, 0.5),
        "oil primary": (0.02, 0.05),
        "fossil gas": (0.02, 0.22),
        "biomass": (0.02, 0.4),
        "green fuel imports": (0.02, 0.57),
        "coal": (0.02, 0.7),
        "curtailment": (0.02, 0.8),
        "renewables": (0.02, 0.9),
        "oil": (0.8, 0.05),
        "gas": (0.2, 0.25),
        "electricity grid": (0.4, 0.7),
        "H2": (0.6, 0.15),
        "battery": (0.55, 0.35),
        "ambient heat": (0.8, 0.9),
        "solar thermal": (0.8, 0.96),
        "heat": (0.9, 0.9),
        "heat demand": (1.0, 0.7),
        "DAC demand": (1.0, 0.6),
        "other demand": (1.0, 0.3),
        "losses": (1.0, 0.6),
    }

    # Sort labels in the order of appearance in positions
    labels = sorted(labels, key=lambda x: positions[x][0])

    nodes = pd.Series({v: i for i, v in enumerate(labels)})

    node_colors = pd.Series(nodes.index.map(colors).fillna("grey"), index=nodes.index)

    # Dummy nodes transparent
    node_colors.loc["dummy1"] = "rgba(0,0,0,0)"
    node_colors.loc["dummy2"] = "rgba(0,0,0,0)"

    x = [positions[v][0] for v in nodes.index]
    y = [positions[v][1] for v in nodes.index]

    link_colors = [
        (
            "rgba{}".format(to_rgba(node_colors[src], alpha=0.7))
            if label != "dummy"
            else "rgba(0,0,0,0)"
        )
        for label, src in zip(connections.label, connections.source)
    ]

    pretty_names = {
        "oil primary": "Fossil Oil<br>({})",
        "fossil gas": "Fossil Gas<br>({})",
        "renewables": "Renewables & Nuclear<br>({})",
        "curtailment": "Curtailment<br>({})",
        "green fuel imports": "Green Fuel Imports<br>({})",
        "gas": "Gas<br>({})",
        "electricity grid": "Electricity Grid<br>({})",
        "H2": "Hydrogen<br>({})",
        "oil": "Oil<br>({})",
        "heat": "Heat<br>({})",
        "other demand": "Other Demand<br>({})",
        "heat demand": "Heat Demand<br>({})",
        "ambient heat": "Ambient Heat<br>({})",
        "solar thermal": "Solar Thermal<br>({})",
        "battery": "Battery<br>({})",
        "biomass": "Biomass<br>({})",
        "coal": "Coal<br>({})",
        "losses": "Losses<br>({})",
        "DAC demand": "Direct air capture<br>({})",
        "dummy1": "",
        "dummy2": "",
    }

    node_values = pd.Series(
        np.maximum(
            connections.groupby("target")
            .value.sum()
            .reindex(nodes.index)
            .fillna(0)
            .values,
            connections.groupby("source")
            .value.sum()
            .reindex(nodes.index)
            .fillna(0)
            .values,
        ),
        index=nodes.index,
    )

    # Round to tens of TWh, as int
    node_values = np.round(node_values, -1).astype(int)

    node_labels = [
        pretty_names.get(v, v + "<br>({})").format(node_values[v]) for v in nodes.index
    ]

    fig = go.Figure(
        go.Sankey(
            arrangement="fixed",  # [snap, nodepad, perpendicular, fixed]
            valuesuffix="TWh",
            valueformat=".1f",
            node=dict(
                pad=pad,
                thickness=4,
                label=node_labels,
                color=node_colors,
                x=x,
                y=y,
                line=dict(width=0),
                align="right",
            ),
            link=dict(
                source=connections.source.map(nodes),
                target=connections.target.map(nodes),
                value=connections.value,
                label=connections.label,
                color=link_colors,
            ),
            textfont=dict(size=10),
        )
    )

    if title is not None:
        fig.update_layout(title=title, font_size=9, title_x=0.5)

    # Remove margin
    fig.update_layout(margin=dict(l=10, r=10, t=30, b=10))

    if fn is not None:
        # Save as pdf
        if fn.endswith(".pdf"):
            fig.write_image(fn, width=400, height=240, engine="kaleido")
        else:
            fig.write_html(fn + ".html")
    else:
        fig.show()

In [8]:
scenarios = [
    ("Y1987_Bb", "Ca-Ia-Eb-Tb_2040_min0.05", "(a) Low CCS, minimum green hydrogen"),
    ("Y1987_Bb", "Ca-Ia-Eb-Tb_2040_max0.05", "(b) Low CCS, maximum green hydrogen"),
    ("Y1987_Bb", "Cc-Ia-Eb-Tb_2040_min0.05", "(c) High CCS, minimum green hydrogen"),
    ("Y1987_Bb", "Cc-Ia-Eb-Tb_2040_max0.05", "(d) High CCS, maximum green hydrogen"),
    ("Y1987_Bb", "Ca-Ia-Eb-Tb_2050_min0.05", "(a) Low CCS, minimum green hydrogen"),
    ("Y1987_Bb", "Ca-Ia-Eb-Tb_2050_max0.05", "(b) Low CCS, maximum green hydrogen"),
    ("Y1987_Bb", "Cc-Ia-Eb-Tb_2050_min0.05", "(c) High CCS, minimum green hydrogen"),
    ("Y1987_Bb", "Cc-Ia-Eb-Tb_2050_max0.05", "(d) High CCS, maximum green hydrogen"),
    ("Y1987_Bb", "Ca-Ia-Eb-Tb_2040", "(a) Low CCS"),
    ("Y1987_Bb", "Cc-Ia-Eb-Tb_2040", "(b) High CCS"),
    ("Y1987_Bb", "Ca-Ia-Eb-Tb_2050", "(a) Low CCS"),
    ("Y1987_Bb", "Cc-Ia-Eb-Tb_2050", "(b) High CCS"),
]

connections = {}

for (S, O, title) in scenarios:
    if not use_cache:
        if copy_networks and os.path.exists(f"../results/{S}/postnetworks/base_s_50_lc1.5__{O}.nc"):
            os.makedirs(f"sankey_networks/{S}", exist_ok=True)
            os.system(
                f"cp ../results/{S}/postnetworks/base_s_50_lc1.5__{O}.nc sankey_networks/{S}/"
            )
        if not os.path.exists(f"sankey_networks/{S}/base_s_50_lc1.5__{O}.nc"):
            print(f"Network not found: sankey_networks/{S}/base_s_50_lc1.5__{O}.nc")
            continue
        n = pypsa.Network(f"sankey_networks/{S}/base_s_50_lc1.5__{O}.nc")
    for losses, dir in [(False, "sankey"), (True, "sankey/with_losses")]:
        if not os.path.exists(f"figures/{dir}"):
            os.makedirs(f"figures/{dir}")
        if not use_cache:
            D, D_pre_agg = prepare_sankey(n, losses=losses, agg_flows=True)
            # Save the connections
            D.to_csv(f"figures/{dir}/{S}_{O}.csv", index=False)
        else:
            D = pd.read_csv(f"figures/{dir}/{S}_{O}.csv")
        connections[(S, O, losses)] = D

        plot_sankey(D, 15000, pad=0, fn=f"figures/{dir}/{S}_{O}.pdf", title=title)

In [6]:
# Find the (S, O) scenarios with the lowest and highest total losses; print them
losses = {}
for (S, O, _) in scenarios:
    if (S, O, True) not in connections:
        continue
    D = connections[(S, O, True)]
    losses[(S, O, True)] = D.loc[D.target == "losses", "value"].sum()

print("Lowest losses:")
min_key = min(losses, key=losses.get)
print(min_key)
print(connections[min_key].loc[connections[min_key].target == "losses"])
print("Total losses:", losses[min_key].round(1))

print()

print("Highest losses:")
max_key = max(losses, key=losses.get)
print(max_key)
print(connections[max_key].loc[connections[max_key].target == "losses"])
print("Total losses:", losses[max_key].round(1))

Lowest losses:
('Y1987_Bb', 'Cc-Ia-Eb-Tb_2040_min0.05', True)
                         label            source  target       value
9            biomass to losses           biomass  losses  199.272886
15  electricity grid to losses  electricity grid  losses  170.201346
22       oil primary to losses       oil primary  losses  153.563152
25        renewables to losses        renewables  losses  362.612606
Total losses: 885.6

Highest losses:
('Y1987_Bb', 'Ca-Ia-Eb-Tb_2040_max0.05', True)
                         label            source  target        value
1                 H2 to losses                H2  losses   379.402930
10           biomass to losses           biomass  losses   258.273024
17  electricity grid to losses  electricity grid  losses  1374.951951
22               gas to losses               gas  losses   107.478014
26       oil primary to losses       oil primary  losses    52.971512
29        renewables to losses        renewables  losses   519.708466
Total losses: 2692.

In [7]:
# For each scenario, print total green hydrogen production
for (S, O, _) in scenarios:
    if (S, O, True) not in connections:
        continue
    D = connections[(S, O, True)]
    green_h2 = D.loc[(D.source == 'electricity grid') & (D.target == 'H2'), 'value'].sum()
    green_h2 = green_h2 / 33.3 # Convert from TWh to Mt H2
    print(f"{S}_{O}: {green_h2:.1f} Mt")

Y1987_Bb_Ca-Ia-Eb-Tb_2040_min0.05: 23.4 Mt
Y1987_Bb_Ca-Ia-Eb-Tb_2040_max0.05: 77.5 Mt
Y1987_Bb_Cc-Ia-Eb-Tb_2040_min0.05: 3.4 Mt
Y1987_Bb_Cc-Ia-Eb-Tb_2040_max0.05: 50.0 Mt
Y1987_Bb_Ca-Ia-Eb-Tb_2050_min0.05: 22.2 Mt
Y1987_Bb_Ca-Ia-Eb-Tb_2050_max0.05: 67.2 Mt
Y1987_Bb_Cc-Ia-Eb-Tb_2050_min0.05: 2.8 Mt
Y1987_Bb_Cc-Ia-Eb-Tb_2050_max0.05: 50.7 Mt
Y1987_Bb_Ca-Ia-Eb-Tb_2040: 32.4 Mt
Y1987_Bb_Cc-Ia-Eb-Tb_2040: 19.5 Mt
Y1987_Bb_Ca-Ia-Eb-Tb_2050: 32.6 Mt
Y1987_Bb_Cc-Ia-Eb-Tb_2050: 13.5 Mt
